# Day 17
## Part 1

This looks tedious. Represent rows and rocks with numpy arrays. Don't use a ndarry as it will keep having to be resized.

In [1]:
from collections import defaultdict
import numpy as np
from itertools import cycle


def rocks():
    rock_strings = """
####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##
"""
    rocks = []
    for rock_string in rock_strings.strip().split("\n\n"):
        rocks.append(
            list(reversed( # Reverse as using a proper y axis
                [
                    np.array([1 if c == "#" else 0 for c in line])
                    for line in rock_string.splitlines()
                ]
            ))
        )
    yield from cycle(rocks)

    
def jets(s):
    yield from cycle(s)

    
def initial_chamber():
    c = defaultdict(lambda: np.array([1, 0, 0, 0, 0, 0, 0, 0, 1]))
    c[0] = np.ones((9,), dtype=int)
    for i in range(1, 5):
        c[i] = np.array([1, 0, 0, 0, 0, 0, 0, 0, 1])
    return c

                    
def height(chamber):
    for h in reversed(sorted(chamber.keys())):
        if chamber[h].sum() > 2:
            return h
    return 0


def draw_chamber(chamber):
    for k in reversed(sorted(chamber)):
        print("".join('#' if x == 1 else '.' for x in chamber[k]))
    print()

In [2]:
def add_rock(rock, chamber, jets):
    x = 3
    y = height(chamber) + 4
    w = len(rock[0])
    h = len(rock)
    for i in range(h):
        chamber[y+i][x:x+w] += rock[i]
    while True:
        # Jet movement
        dx = -1 if next(jets) == '<' else 1
        new_x = x + dx
        new_chamber = {}
        for i in range(h):
            new_chamber[y+i] = chamber[y+i].copy()
            new_chamber[y+i][x:x+w] -= rock[i]
        for i in range(h):
            new_chamber[y+i][new_x:new_x+w] += rock[i]
        if not any((row > 1).any() for row in new_chamber.values()):
            for k in new_chamber:
                chamber[k] = new_chamber[k]
                x = new_x
#         print(f"{dx = }")
#         draw_chamber(chamber)
                
        # Down movement
        new_chamber = {}
        new_y = y - 1
        for i in range(h):
            new_chamber[y+i] = chamber[y+i].copy()
            new_chamber[y+i][x:x+w] -= rock[i]
        new_chamber[y-1] = chamber[y-1].copy()
        for i in range(h):
            new_chamber[y-1+i][x:x+w] += rock[i]
        if any((row > 1).any() for row in new_chamber.values()):
            return chamber
        else:
            for k in new_chamber:
                chamber[k] = new_chamber[k]
                y = new_y
#         print("Down")
#         draw_chamber(chamber)
                
def part_1(s):
    j = jets(s)
    chamber = initial_chamber()
    for rock, _ in zip(rocks(), range(2022)):
        chamber = add_rock(rock, chamber, j)
#         draw_chamber(chamber)
#         print()
    return height(chamber)

In [3]:
test_data = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
part_1(test_data)

3068

In [5]:
data = open("input").read().strip()
part_1(data)

3193

## Part 2

Need to find a cycle. Represent the last 200 rows as a binary number and see where it repeats. Use fiddly modular arithmetic to calculate the height after a very large number of rocks.

In [58]:
from itertools import count

def find_loop(data):
    j = jets(data)
    chamber = initial_chamber()
    seen = {}
    for rock, i in zip(rocks(), count(1)):
        if i % 10000 == 0:
            print(i)
        chamber = add_rock(rock, chamber, j)
        h = height(chamber)
        s = ""
        if h > 200:
            for u in range(200):
                s = s + "".join(str(x) for x in chamber[h-u][1:-1])
            if s in seen:
                return (seen[s], (i, h))
            seen[s] = (i, h)
        
def part_2(data):
    (i1, h1), (i2, h2) = find_loop(data)
    mod = i2 - i1
    q, r = divmod(1000000000000 - i1, mod)
    chamber = initial_chamber()
    j = jets(data)
    for rock, _ in zip(rocks(), range(i1 + r)):
        chamber = add_rock(rock, chamber, j)
    h = height(chamber)
    return h1 + q * (h2 - h1) + (h - h1)

part_2(test_data)

1514285714288

In [59]:
part_2(data)

1577650429835